In [34]:
# Importing Libraries
import requests, time, random
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver import ActionChains
import pandas as pd
import pprint
import time
import datetime
from datetime import date
import math
from numpy import inf
from datetime import timedelta
import numpy as np
from dateutil.relativedelta import relativedelta
from selenium.webdriver.support.ui import Select
from datetime import datetime

#Libraries for SQL connection
import mysql.connector
import pymysql
from sqlalchemy import create_engine

In [35]:
# Initializing a Chrome Browser using Selenium
browser = webdriver.Chrome(r'C:\Users\madhu\Documents\Mugilan\Renewable Energy\chromedriver_win32\chromedriver.exe')

In [36]:
# Access webpage
browser.get('https://cea.nic.in/daily-renewable-generation-report/?lang=en')

browser.maximize_window()

time.sleep(30)

In [37]:
# Select ALl Records per page

select = Select(browser.find_element_by_name('table_length'))

select.select_by_value('-1')

In [38]:
# Set Min and Max Date
max_dt = (date.today() + relativedelta(days=1)).strftime("%m/%d/%Y")
min_dt = (date.today() - relativedelta(months=1)).strftime("%m/01/%Y")

print(max_dt)

#Set Min Date
min_date = browser.find_element_by_name('min')
min_date.clear()
min_date.send_keys(min_dt)

#Set Max Date
max_date = browser.find_element_by_name('max')
max_date.clear()
max_date.send_keys(max_dt)

07/30/2021


In [39]:
#Extract Latest Report Link

report_down_link_df = pd.DataFrame()

try:
    for all_rep in browser.find_elements_by_class_name('dataTable.no-footer'):
        for rep in all_rep.find_elements_by_tag_name('tbody'):
            for latest_rep in rep.find_elements_by_tag_name('tr'):
                report_dt = latest_rep.find_element_by_class_name('sorting_1').text
                for latest_rep_xl in latest_rep.find_elements_by_tag_name('a'):
                    if latest_rep_xl.get_attribute('href').endswith('.xlsx'):
                        latest_report_link = latest_rep_xl.get_attribute('href')
                        df_len = len(report_down_link_df)
                        report_down_link_df.loc[df_len,'download_link'] = latest_report_link
                        report_down_link_df.loc[df_len,'report_date'] = report_dt
                        
except:
    pass

report_down_link_df = report_down_link_df.drop_duplicates().reset_index(drop = True)

report_down_link_df['report_date'] = pd.to_datetime(report_down_link_df['report_date'])

In [70]:
# Connect to the database
mydb = mysql.connector.connect(
  host="mysql-41080-0.cloudclusters.net",
  port=18738,
  db='energy',
  user="test",
  password="test1234"
)

# create cursor
cursor=mydb.cursor()

In [41]:
# Checking Max Date from the Master Table

cursor.execute("SELECT count(distinct report_date) as report_count FROM energy_state_level")

report_cnt = cursor.fetchall()

for x in report_cnt:
    report_count = int(x[0])
    print(report_count)

if report_count > 0:
    
    cursor.execute("SELECT max(report_date) as max_report_date FROM energy_state_level")

    max_report_dt = cursor.fetchall()

    for x in max_report_dt:
        max_report_date = x[0]
        
else:
    max_report_date = '01-01-2020'

max_report_date = pd.to_datetime(max_report_date)

print(max_report_date)

report_down_link_df_bkp = report_down_link_df.copy()

report_down_link_df = report_down_link_df[report_down_link_df['report_date'] > max_report_date]

report_down_link_df = report_down_link_df.reset_index(drop = True)

48
2021-07-18 00:00:00


In [43]:
# Extract Data from Reports

master_report_df = pd.DataFrame()

for i in range(len(report_down_link_df)):
    
    report_df = pd.DataFrame()
    
    report_link = report_down_link_df.loc[i,'download_link']
    report_date = report_down_link_df.loc[i,'report_date']
    
    report_df = pd.read_excel(report_link, sheet_name= 0,skiprows=5)
    
    report_df = report_df.dropna(thresh=8)
    
    report_df = report_df.dropna(how='all',axis=1)
    
    report_df.columns = ['state','wind','solar','others','total','wind_cum','solar_cum','others_cum','total_cum']
    
    report_df['report_date'] = report_date.strftime("%Y-%m-%d")
    
    report_df['state'] = report_df['state'].str.split('/',expand=True)[1]
    
    report_df['state'] = report_df['state'].str.strip()
    
    master_report_df = pd.concat([master_report_df,report_df])
    
    master_report_df = master_report_df.reset_index(drop = True)

In [45]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in master_report_df.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in master_report_df.iterrows():
    sql = "INSERT INTO `energy_state_level` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    mydb.commit()

In [46]:
# Access webpage
browser.get('https://www.renewablesindia.in/')

time.sleep(30)

In [80]:
#Extracting CO2 Emission

co2_class = browser.find_element_by_class_name('col-xs-12.col-sm-12')

co2 = co2_class.find_element_by_class_name('card-heading-1').text

co2 = co2.split()[0].strip()

r_date = browser.find_element_by_class_name('card-date-1').text

r_date_format = pd.to_datetime(r_date).strftime("%Y-%m-%d")

In [89]:
# Connect to the database
mydb = mysql.connector.connect(
  host="mysql-41080-0.cloudclusters.net",
  port=18738,
  db='energy',
  user="test",
  password="test1234"
)

# create cursor
cursor=mydb.cursor()

In [83]:
# Writing to DB

sql = "TRUNCATE TABLE daily_overall_summary"

cursor.execute(sql)

sql = "INSERT INTO `daily_overall_summary` (`co`, `report_date`, `report_date_format`) VALUES (%s, %s, %s)"

cursor.execute(sql, (co2, r_date, r_date_format))

mydb.commit()

In [86]:
# Extract Latest ISGS Data

for i in range(1):
    
    report_link = report_down_link_df.loc[i,'download_link']
    report_date = report_down_link_df.loc[i,'report_date']
    
    isgs_report_df = pd.read_excel(report_link, sheet_name= 1,skiprows=5)
    
    isgs_report_df = isgs_report_df.dropna(thresh=8)
    
    isgs_report_df = isgs_report_df.dropna(how='all',axis=1)
    
    isgs_report_df.columns = ['station','state','sector','owner','type','installed','actual','cumulative']
    
    isgs_report_df = isgs_report_df[['state','sector','type','installed','actual']]

In [90]:
# Writing to DB

sql = "TRUNCATE TABLE isgs_data"

cursor.execute(sql)

# creating column list for insertion
cols = "`,`".join([str(i) for i in isgs_report_df.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in isgs_report_df.iterrows():
    sql = "INSERT INTO `isgs_data` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    mydb.commit()